Trabalho final de Redes
=============================



## Introdução



Muitos problemas de otimização com relevância científica têm uma ou mais `restrições` que devem ser levadas em consideração na hora de resolver o problema.

Lembra do `problema da mochila` que vimos em Lógica Computacional? Era um problema de otimização onde queríamos maximizar o valor dos itens colocados na mochila enquanto observávamos a restrição do peso total dos itens (do contrário, a mochila rasgava).

Uma forma de considerar essas restrições nos problemas é aplicando uma `penalidade` na função objetivo.

Vamos pensar como seria essa penalidade no problema da mochila: a função objetivo é maximizar o valor dos itens na mochila, então é um problema de maximização. A função objetivo pode ser a soma dos itens da mochila. Se fosse só isso, teríamos

$$
f = \sum_{i, i \in \mathrm{mochila}}\mathrm{valor}(i)
$$

No entanto, apenas essa função não resolve o problema! Precisamos levar em consideração o limite de peso da mochila! Para isso, penalizamos a função objetivo levando em consideração essa restrição:

$f=\begin{cases}
0,01 & \textrm{se peso > limite da mochila}\\
\sum_{i,i\in\mathrm{mochila}}(\mathrm{valor}(i)) & \textrm{se peso} \leq \textrm{limite da mochila}
\end{cases}$

Agora finalmente podemos seguir em frente e resolver o problema.



## Reflexões



Se usarmos a equação de $f$ acima, qual será o valor de $f$ caso não exista uma solução para um certo problema da mochila?

Na equação de $f$ acima nós usamos o valor zero para indicar que uma restrição do problema não foi satisfeita. Você consegue pensar em outra estratégia para penalizar soluções inválidas?



## Objetivo



Encontrar uma solução para o problema da mochila usando algoritmos genéticos. Considere que existem 10 itens diferentes (com pesos e valores diferentes) disponíveis para serem escolhidos.



## Descrição do problema



No problema da mochila você tem um número $n$ de itens disponíveis, cada um com um peso e um valor associado. Sua mochila tem a capacidade de carregar um número $p$ de quilogramas, sendo que mais que isso faz com que sua mochila rasgue e todos os itens dentro dela caiam no chão e se quebrem de maneira catastrófica (indesejado). Sua tarefa é encontrar um conjunto de itens (considerando os $n$ disponíveis) que maximize o valor contido dentro da mochila, porém que tenham um peso dentro da capacidade da mesma.



## Importações



In [1]:
import random

from funcoes import computa_maternidade 
from funcoes import funcao_objetivo_pop_maternidade
from funcoes import populacao_cb as cria_populacao_inicial
from funcoes import selecao_roleta_max as funcao_selecao
from funcoes import cruzamento_ponto_simples as funcao_cruzamento
from funcoes import mutacao_cb as funcao_mutacao
from funcoes import funcao_objetivo_pop_maternidade
from funcoes import funcao_objetivo_maternidade

## Códigos e discussão



In [52]:
### CONSTANTES

# relacionadas à busca
TAMANHO_POP = 20
NUM_GERACOES = 100
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.05

# relacionadas ao problema a ser resolvido
LIMITE_DE_PESO = 5

MULHERES = {
    'mulher1': {
        'hemorragia': 1,
        'idade gestacional': 10,
        'dor': 5,
        'meows': 2,
        'sinais de choque': 1,
        'leito': 1
    },
    'mulher2': {
        'hemorragia': 3,
        'idade gestacional': 20,
        'dor': 8,
        'meows': 4,
        'sinais de choque': 3,
        'leito': 1
    },
    'mulher3': {
        'hemorragia': 2,
        'idade gestacional': 15,
        'dor': 3,
        'meows': 3,
        'sinais de choque': 2,
        'leito': 1
    },
    'mulher4': {
        'hemorragia': 4,
        'idade gestacional': 5,
        'dor': 1,
        'meows': 1,
        'sinais de choque': 1,
        'leito': 1
    },
    'mulher5': {
        'hemorragia': 1,
        'idade gestacional': 30,
        'dor': 9,
        'meows': 4,
        'sinais de choque': 2,
        'leito': 1
    },
    'mulher6': {
        'hemorragia': 4,
        'idade gestacional': 35,
        'dor': 2,
        'meows': 3,
        'sinais de choque': 3,
        'leito': 1
    },
    'mulher7': {
        'hemorragia': 3,
        'idade gestacional': 25,
        'dor': 6,
        'meows': 1,
        'sinais de choque': 1,
        'leito': 1
    },
    'mulher8': {
        'hemorragia': 2,
        'idade gestacional': 18,
        'dor': 4,
        'meows': 2,
        'sinais de choque': 2,
        'leito': 1
    },
    'mulher9': {
        'hemorragia': 1,
        'idade gestacional': 7,
        'dor': 3,
        'meows': 4,
        'sinais de choque': 1,
        'leito': 1
    },
    'mulher10': {
        'hemorragia': 4,
        'idade gestacional': 32,
        'dor': 9,
        'meows': 1,
        'sinais de choque': 2,
        'leito': 1
    },
    'mulher11': {
        'hemorragia': 3,
        'idade gestacional': 14,
        'dor': 8,
        'meows': 3,
        'sinais de choque': 3,
        'leito': 1
    },
    'mulher12': {
        'hemorragia': 1,
        'idade gestacional': 23,
        'dor': 2,
        'meows': 2,
        'sinais de choque': 1,
        'leito': 1
    },
    'mulher13': {
        'hemorragia': 2,
        'idade gestacional': 11,
        'dor': 5,
        'meows': 4,
        'sinais de choque': 2,
        'leito': 1
    },
    'mulher14': {
        'hemorragia': 4,
        'idade gestacional': 37,
        'dor': 1,
        'meows': 3,
        'sinais de choque': 3,
        'leito': 1
    },
    'mulher15': {
        'hemorragia': 3,
        'idade gestacional': 28,
        'dor': 7,
        'meows': 1,
        'sinais de choque': 1,
        'leito': 1
    },
    'mulher16': {
        'hemorragia': 2,
        'idade gestacional': 17,
        'dor': 4,
        'meows': 2,
        'sinais de choque': 2,
        'leito': 1
    },
    'mulher17': {
        'hemorragia': 1,
        'idade gestacional': 6,
        'dor': 3,
        'meows': 4,
        'sinais de choque': 1,
        'leito': 1
    },
    'mulher18': {
        'hemorragia': 4,
        'idade gestacional': 31,
        'dor': 9,
        'meows': 1,
        'sinais de choque': 2,
        'leito': 1
    },
    'mulher19': {
        'hemorragia': 3,
        'idade gestacional': 13,
        'dor': 8,
        'meows': 3,
        'sinais de choque': 3,
        'leito': 1
    },
    'mulher20': {
        'hemorragia': 1,
        'idade gestacional': 22,
        'dor': 2,
        'meows': 2,
        'sinais de choque': 1,
        'leito': 1
    },
    'mulher21': {
        'hemorragia': 2,
        'idade gestacional': 9,
        'dor': 5,
        'meows': 4,
        'sinais de choque': 2,
        'leito': 1
    },
    'mulher22': {
        'hemorragia': 4,
        'idade gestacional': 38,
        'dor': 1,
        'meows': 3,
        'sinais de choque': 3,
        'leito': 1
    },
    'mulher23': {
        'hemorragia': 3,
        'idade gestacional': 29,
        'dor': 7,
        'meows': 1,
        'sinais de choque': 1,
        'leito': 1
    },
    'mulher24': {
        'hemorragia': 2,
        'idade gestacional': 19,
        'dor': 4,
        'meows': 2,
        'sinais de choque': 2,
        'leito': 1
    },
    'mulher25': {
        'hemorragia': 1,
        'idade gestacional': 8,
        'dor': 3,
        'meows': 4,
        'sinais de choque': 1,
        'leito': 1
    },
    'mulher26': {
        'hemorragia': 4,
        'idade gestacional': 33,
        'dor': 9,
        'meows': 1,
        'sinais de choque': 2,
        'leito': 1
    },
    'mulher27': {
        'hemorragia': 3,
        'idade gestacional': 16,
        'dor': 8,
        'meows': 3,
        'sinais de choque': 3,
        'leito': 1
    }
}

NUM_MULHERES = len(MULHERES)
ORDEM_DOS_NOMES = list(sorted(MULHERES.keys()))


In [53]:
# Funções locais

def funcao_objetivo_pop(populacao):
    return funcao_objetivo_pop_maternidade(
        populacao, MULHERES, LIMITE_DE_PESO, ORDEM_DOS_NOMES
    )

In [82]:
# Busca por algoritmo genético

populacao = cria_populacao_inicial(TAMANHO_POP, NUM_MULHERES)

# variaveis para o hall da fama
melhor_fitness_ja_visto = -float("inf")
melhor_individuo_ja_visto = [0] * NUM_MULHERES

for n in range(NUM_GERACOES):

    # Seleção
    fitness = funcao_objetivo_pop(populacao)
    populacao = funcao_selecao(populacao, fitness)

    # Cruzamento
    pais = populacao[0::2]
    maes = populacao[1::2]

    contador = 0

    for pai, mae in zip(pais, maes):
        if random.random() <= CHANCE_CRUZAMENTO:
            filho1, filho2 = funcao_cruzamento(pai, mae)
            populacao[contador] = filho1
            populacao[contador + 1] = filho2

        contador = contador + 2

    # Mutação
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO:
            individuo = populacao[n]
            populacao[n] = funcao_mutacao(individuo)

    # melhor individuo já visto até agora (hall da fama)
    fitness = funcao_objetivo_pop(populacao)
    maior_fitness = max(fitness)
    posicao = fitness.index(maior_fitness)
    individuo = populacao[posicao].copy()
    valor, peso = computa_maternidade(individuo, MULHERES, ORDEM_DOS_NOMES)
    if maior_fitness > melhor_fitness_ja_visto and peso <= LIMITE_DE_PESO: # nao queremos que ele nos de individuos que nao resolvam o problema
        melhor_fitness_ja_visto = maior_fitness
        melhor_individuo_ja_visto = individuo
        print(f"Maior valor: {valor} | Peso: {peso}")


# reportando o melhor individuo encontrado
print()
print("Você deve priorizar as seguintes pessoas grávidas:")
for pega_ou_nao, item in zip(melhor_individuo_ja_visto, ORDEM_DOS_NOMES):
    if pega_ou_nao == 1:
        print("+", item)
print()
valor_total, peso_total = computa_maternidade(
    melhor_individuo_ja_visto, MULHERES, ORDEM_DOS_NOMES
)
print(
    f"Com isso, de acordo a disponibilidade dos {peso_total} leitos e com os parâmetros avaliados" 
    f" as pessoas gestantes que serão atendidas com maior prioridade serão:{valor_total}. "
)

Maior valor: 1311 | Peso: 5
Maior valor: 1497 | Peso: 5

Você deve priorizar as seguintes pessoas grávidas:
+ mulher14
+ mulher21
+ mulher27
+ mulher3
+ mulher7

Com isso, de acordo a disponibilidade dos 5 leitos e com os parâmetros avaliados as pessoas gestantes que serão atendidas com maior prioridade serão:1497. 


## Conclusão



## Playground



In [5]:
,
    'mulher28': {
        'hemorragia': 1,
        'idade gestacional': 25,
        'dor': 2,
        'meows': 2,
        'sinais de choque': 1,
        'leito': 1
    },
    'mulher29': {
        'hemorragia': 2,
        'idade gestacional': 10,
        'dor': 5,
        'meows': 4,
        'sinais de choque': 2,
        'leito': 1
    },
    'mulher30': {
        'hemorragia': 4,
        'idade gestacional': 35,
        'dor': 1,
        'meows': 3,
        'sinais de choque': 3,
        'leito': 1
    },
    'mulher31': {
        'hemorragia': 3,
        'idade gestacional': 26,
        'dor': 7,
        'meows': 1,
        'sinais de choque': 1,
        'leito': 1
    },
    'mulher32': {
        'hemorragia': 2,
        'idade gestacional': 18,
        'dor': 4,
        'meows': 2,
        'sinais de choque': 2,
        'leito': 1
    },
    'mulher33': {
        'hemorragia': 1,
        'idade gestacional': 7,
        'dor': 3,
        'meows': 4,
        'sinais de choque': 1,
        'leito': 1
    },
    'mulher34': {
        'hemorragia': 4,
        'idade gestacional': 32,
        'dor': 9,
        'meows': 1,
        'sinais de choque': 2,
        'leito': 1
    },
    'mulher35': {
        'hemorragia': 3,
        'idade gestacional': 14,
        'dor': 8,
        'meows': 3,
        'sinais de choque': 3,
        'leito': 1
    },
    'mulher36': {
        'hemorragia': 1,
        'idade gestacional': 23,
        'dor': 2,
        'meows': 2,
        'sinais de choque': 1,
        'leito': 1
    },
    'mulher37': {
        'hemorragia': 2,
        'idade gestacional': 11,
        'dor': 5,
        'meows': 4,
        'sinais de choque': 2,
        'leito': 1
    },
    'mulher38': {
        'hemorragia': 4,
        'idade gestacional': 37,
        'dor': 1,
        'meows': 3,
        'sinais de choque': 3,
        'leito': 1
    },
    'mulher39': {
        'hemorragia': 3,
        'idade gestacional': 28,
        'dor': 7,
        'meows': 1,
        'sinais de choque': 1,
        'leito': 1
    },
    'mulher40': {
        'hemorragia': 2,
        'idade gestacional': 17,
        'dor': 4,
        'meows': 2,
        'sinais de choque': 2,
        'leito': 1
    },
    'mulher41': {
        'hemorragia': 1,
        'idade gestacional': 6,
        'dor': 3,
        'meows': 4,
        'sinais de choque': 1,
        'leito': 1
    },
    'mulher42': {
        'hemorragia': 4,
        'idade gestacional': 31,
        'dor': 9,
        'meows': 1,
        'sinais de choque': 2,
        'leito': 1
    },
    'mulher43': {
        'hemorragia': 3,
        'idade gestacional': 13,
        'dor': 8,
        'meows': 3,
        'sinais de choque': 3,
        'leito': 1
    },
    'mulher44': {
        'hemorragia': 1,
        'idade gestacional': 22,
        'dor': 2,
        'meows': 2,
        'sinais de choque': 1,
        'leito': 1
    },
    'mulher45': {
        'hemorragia': 2,
        'idade gestacional': 9,
        'dor': 5,
        'meows': 4,
        'sinais de choque': 2,
        'leito': 1
    },
    'mulher46': {
        'hemorragia': 4,
        'idade gestacional': 38,
        'dor': 1,
        'meows': 3,
        'sinais de choque': 3,
        'leito': 1
    },
    'mulher47': {
        'hemorragia': 3,
        'idade gestacional': 29,
        'dor': 7,
        'meows': 1,
        'sinais de choque': 1,
        'leito': 1
    },
    'mulher48': {
        'hemorragia': 2,
        'idade gestacional': 19,
        'dor': 4,
        'meows': 2,
        'sinais de choque': 2,
        'leito': 1
    },
    'mulher49': {
        'hemorragia': 1,
        'idade gestacional': 8,
        'dor': 3,
        'meows': 4,
        'sinais de choque': 1,
        'leito': 1
    },
    'mulher50': {
        'hemorragia': 4,
        'idade gestacional': 33,
        'dor': 9,
        'meows': 1,
        'sinais de choque': 2,
        'leito': 1
    }

IndentationError: unexpected indent (Temp/ipykernel_24332/1171548724.py, line 2)